# Data 690 Practical AI: Final Project
Jessica Conroy
## Notebook 3: Run Detection App
Contents:
- <b>Notebook 1</b> in this series collects the dataset for the project. 
- <b>Notebook 2</b> takes the next step by preparing and transforming the data for use in a pytorch model. It includes the training of the model with updatable constants for testing different parameters and configurations.
- <b>Notebook 3</b> This is the code used in the App.py script below, but with additional documentation added
- <b>App.py</b> represents the 3rd piece of the project which will launch the sign language detector. 

### Sources
https://docs.opencv.org/4.x/dd/d43/tutorial_py_video_display.html

https://learnopencv.com/cropping-an-image-using-opencv/

https://www.analyticsvidhya.com/blog/2021/07/building-a-hand-tracking-system-using-opencv/


In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torchvision.models import resnet50
from torch.nn import CrossEntropyLoss
from torch.nn import Dropout
from torch.nn import Identity
from torch.nn import Linear
from torch.nn import Module
from torch.nn import ReLU
from torch.nn import Sequential
from torch.nn import Sigmoid
from torch.nn import Flatten
import torch.nn as nn
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import cv2
import glob
import numpy
import random
import xml.etree.ElementTree as ET

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import os

import time

os.getcwd()

'C:\\Users\\15856\\Data 690 AI\\Final Project'

In [2]:
train_data_path = './data/collectedimgs/train' 
test_data_path = './data/collectedimgs/test'


train_image_paths = []
train_annotation_paths = []
test_image_paths = []
test_annotations_paths = []

train_annotation_paths = glob.glob(train_data_path+"/*.xml")
train_annotation_paths = sorted(train_annotation_paths)
train_image_paths = glob.glob(train_data_path+"/*.jpg")
train_image_paths = sorted(train_image_paths)

test_annotations_paths = glob.glob(test_data_path+"/*.xml")
test_annotations_paths = sorted(test_annotations_paths)
test_image_paths = glob.glob(test_data_path+"/*.jpg")
test_image_paths = sorted(test_image_paths)

classes = []
for path in train_image_paths:
    filename = path.split('\\')[-1]
    class_name = filename.split('.')[0]
    if class_name == 'ThankYou':
        class_name = 'thank you'
    if class_name in classes:
        continue
    else:
        classes.append(class_name)

print(classes)

['thank you', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'hello', 'i', 'j', 'k', 'l', 'm', 'my', 'n', 'name', 'no', 'o', 'p', 'project', 'q', 'r', 's', 't', 'this', 'u', 'v', 'w', 'x', 'y', 'yes', 'z']


In [3]:
#Uncomment to select only a few classes to train

my_classes = ['ThankYou', 'hello','my', 'name', 'project','this', 'j', 'e', 's']

train_annotation_paths_lite = []
test_annotation_paths_lite = []

train_image_paths_lite = []
test_image_paths_lite = []

for i,path in enumerate(train_image_paths):
    filename = path.split('\\')[-1]
    class_name = filename.split('.')[0]
    for CN in my_classes:
        if CN == class_name:
            train_image_paths_lite.append(path)
            train_annotation_paths_lite.append(train_annotation_paths[i])

for i,path in enumerate(test_image_paths):
    filename = path.split('\\')[-1]
    class_name = filename.split('.')[0]
    for CN in my_classes:
        if CN == class_name:
            test_image_paths_lite.append(path)
            test_annotation_paths_lite.append(test_annotations_paths[i])

classes = []
for path in train_annotation_paths_lite:
    filename = path.split('\\')[-1]
    class_name = filename.split('.')[0]
    if class_name == 'ThankYou':
        class_name = 'thank you'
    if class_name in classes:
        continue
    else:
        classes.append(class_name)
        
print(classes)

['thank you', 'e', 'hello', 'j', 'my', 'name', 'project', 's', 'this']


In [4]:
idx_to_class = {i:j for i, j in enumerate(classes)}
class_to_idx = {value:key for key,value in idx_to_class.items()}
idx_to_class

{0: 'thank you',
 1: 'e',
 2: 'hello',
 3: 'j',
 4: 'my',
 5: 'name',
 6: 'project',
 7: 's',
 8: 'this'}

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

In [6]:
class ObjectClassifier(Module):
    def __init__(self, baseModel, numClasses):
        super(ObjectClassifier, self).__init__()
        # initialize the base model and the number of classes
        self.baseModel = baseModel
        self.numClasses = numClasses

        # build the classifier head to predict the class labels
        self.classifier = Sequential(
            Linear(2048, 512),
            ReLU(),
            Dropout(),
            Linear(512, 512),
            ReLU(),
            Dropout(),
            Linear(512, self.numClasses)
            )
        # set the classifier of our base model to produce outputs
        # from the last convolution block
        self.baseModel.fc = Identity()
    def forward(self, x):
    # pass the inputs through the base model and then obtain
    # predictions from two different branches of the network
        features = self.baseModel(x)
        classLogits = self.classifier(features)
    # return the outputs as a tuple
        return (classLogits)

In [7]:
resnet = resnet50(pretrained=True)
# rcnn = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# freeze all ResNet50 layers so they will *not* be updated during the
# training process
for param in resnet.parameters():
    param.requires_grad = False
    
ObjectClassifier = ObjectClassifier(resnet, len(idx_to_class))
# ObjectClassifier.load_state_dict(torch.load("classifier_200epochs.pth", map_location=torch.device('cpu')))


In [8]:
checkpoint_dict = torch.load('classification_200epochs_croppedTI_checkpoint (1).pth', map_location=device)
ObjectClassifier.load_state_dict(checkpoint_dict['model_state_dict'])
ObjectClassifier.eval()

ObjectClassifier(
  (baseModel): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(

In [19]:
Transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD)
])


import cv2
import mediapipe as mp

mphands = mp.solutions.hands
hands = mphands.Hands()
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    
    h, w, c = frame.shape
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(framergb)
    hand_landmarks = result.multi_hand_landmarks

    if hand_landmarks:
        if len(hand_landmarks) == 2: #When there are two hands in the view, I want a single box containing both hands
            x_max = 0
            y_max = 0
            x_min = w
            y_min = h
            combined = []
            for handLMs in hand_landmarks:
                for lm in handLMs.landmark: #create combined list of hand landmarks
                    combined.append(lm)
            for lm in combined:
                x, y = int(lm.x * w), int(lm.y * h)
                if x > x_max:
                    x_max = x
                if x < x_min:
                    x_min = x
                if y > y_max:
                    y_max = y
                if y < y_min:
                    y_min = y
                    
            #Crop based on identified hands and pas through model for prediction
            
            try:
                image = frame[x_max+10:y_max+10, x_min-10:y_min-10] #crop to just hands
    #             print(image)
    #             try:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, (224, 224))
                image = Transform(image)
                image = image.unsqueeze(0)
                print(image)
                # determine the class label with the largest predicted
                # probability
                prediction = ObjectClassifier(image)
    #             print(prediction)
                prediction = torch.nn.Softmax(dim=1)(prediction)
    #             print(prediction)
                i = prediction.argmax(dim=-1).cpu()
                label = idx_to_class[i.item()]
            except:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, (224, 224))
                image = Transform(image)
                image = image.unsqueeze(0)
#                 print(image)
                # determine the class label with the largest predicted
                # probability
                prediction = ObjectClassifier(image)
    #             print(prediction)
                prediction = torch.nn.Softmax(dim=1)(prediction)
    #             print(prediction)
                i = prediction.argmax(dim=-1).cpu()
                label = idx_to_class[i.item()]
                
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                #https://stackoverflow.com/questions/56108183/python-opencv-cv2-drawing-rectangle-with-text
            cv2.putText(frame, label, (x_min, y_min-10),cv2.FONT_HERSHEY_COMPLEX, .5, (0,0,0),1)
#             mp_drawing.draw_landmarks(frame, handLMs, mphands.HAND_CONNECTIONS)
        else:
            for handLMs in hand_landmarks:
                x_max = 0
                y_max = 0
                x_min = w
                y_min = h
                for lm in handLMs.landmark:
                    x, y = int(lm.x * w), int(lm.y * h)
                    if x > x_max:
                        x_max = x
                    if x < x_min:
                        x_min = x
                    if y > y_max:
                        y_max = y
                    if y < y_min:
                        y_min = y
                        
                #Crop based on identified hands and pas through model for prediction
                image = frame[x_max+10:y_max+10, x_min-10:y_min-10] #crop to just hands
                print(image)
                try:
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    image = cv2.resize(image, (224, 224))
                    image = Transform(image)
                    image = image.unsqueeze(0)
                #     print(image)
                    # determine the class label with the largest predicted
                    # probability
                    prediction = ObjectClassifier(image)
        #             print(prediction)
                    prediction = torch.nn.Softmax(dim=1)(prediction)
        #             print(prediction)
                    i = prediction.argmax(dim=-1).cpu()
                    label = idx_to_class[i.item()]
                except:
                    label = 'No Sign'
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                #https://stackoverflow.com/questions/56108183/python-opencv-cv2-drawing-rectangle-with-text
                cv2.putText(frame, label, (x_min, y_min-10),cv2.FONT_HERSHEY_COMPLEX, .5, (0,255,0),1)
#                 mp_drawing.draw_landmarks(frame, handLMs, mphands.HAND_CONNECTIONS)
    # show the output image 
    cv2.imshow("Output", frame)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[[[ 79  66  52]
  [ 85  73  56]
  [ 88  76  58]
  ...
  [136 154 164]
  [135 155 167]
  [130 152 166]]

 [[ 73  65  51]
  [ 75  67  50]
  [ 77  71  49]
  ...
  [130 152 168]
  [132 153 173]
  [131 153 174]]

 [[ 75  67  52]
  [ 75  67  50]
  [ 79  72  53]
  ...
  [129 153 174]
  [130 154 176]
  [129 153 176]]

 ...

 [[ 38  46  63]
  [ 49  56  74]
  [ 57  64  85]
  ...
  [ 78  71 103]
  [ 78  70 100]
  [ 78  72 100]]

 [[ 49  54  71]
  [ 49  56  74]
  [ 53  60  80]
  ...
  [ 78  70 105]
  [ 76  68 100]
  [ 76  69  97]]

 [[ 50  55  72]
  [ 49  56  74]
  [ 49  55  77]
  ...
  [ 76  68 101]
  [ 77  68 101]
  [ 76  68 101]]]
[[[ 74  72  49]
  [ 72  74  46]
  [ 70  73  44]
  ...
  [118 117 103]
  [111 110  95]
  [109 109  90]]

 [[ 76  77  55]
  [ 76  78  52]
  [ 74  75  50]
  ...
  [127 122 104]
  [118 115  97]
  [113 112  94]]

 [[ 75  76  54]
  [ 76  78  52]
  [ 80  79  53]
  ...
  [127 122 104]
  [120 118 100]
  [117 117  99]]

 ...

 [[ 41  42  33]
  [

[[[ 98 105 118]
  [142 149 167]
  [138 146 168]
  ...
  [127 142 185]
  [133 148 192]
  [146 161 202]]

 [[ 78  84 100]
  [137 144 166]
  [141 147 170]
  ...
  [126 141 183]
  [143 159 199]
  [151 167 207]]

 [[ 69  75  91]
  [129 135 158]
  [143 149 173]
  ...
  [115 131 171]
  [143 159 199]
  [151 168 206]]

 ...

 [[ 37  39  20]
  [ 33  37  18]
  [ 32  36  20]
  ...
  [184 188 199]
  [184 186 198]
  [190 189 203]]

 [[ 35  38  19]
  [ 33  37  18]
  [ 31  35  19]
  ...
  [184 188 199]
  [182 184 195]
  [187 189 200]]

 [[ 38  40  23]
  [ 39  40  27]
  [ 37  38  28]
  ...
  [184 187 201]
  [187 189 200]
  [190 190 200]]]
[[[ 78  88 100]
  [129 138 160]
  [138 147 178]
  ...
  [142 154 191]
  [155 168 203]
  [161 174 207]]

 [[ 71  83  95]
  [119 132 156]
  [132 144 173]
  ...
  [144 153 191]
  [158 169 204]
  [163 176 209]]

 [[ 74  83 100]
  [115 128 150]
  [130 143 171]
  ...
  [144 156 193]
  [154 166 201]
  [161 174 207]]

 ...

 [[ 41  45  27]
  [ 42  46  28]
  [ 38  43  23]
  ..

[[[ 85  82  79]
  [ 81  79  79]
  [ 78  77  76]
  ...
  [150 171 201]
  [119 139 173]
  [105 123 163]]

 [[ 86  84  81]
  [ 79  77  77]
  [ 81  78  77]
  ...
  [161 181 211]
  [125 145 178]
  [107 126 163]]

 [[ 83  82  79]
  [ 80  78  78]
  [ 79  77  76]
  ...
  [166 186 216]
  [127 148 181]
  [111 132 169]]

 ...

 [[ 37  37  28]
  [ 38  39  27]
  [ 38  39  26]
  ...
  [184 190 214]
  [187 193 218]
  [190 195 220]]

 [[ 39  40  28]
  [ 38  39  27]
  [ 41  41  29]
  ...
  [185 186 211]
  [190 192 216]
  [194 196 218]]

 [[ 40  39  32]
  [ 38  38  31]
  [ 40  41  31]
  ...
  [181 183 205]
  [186 187 210]
  [189 191 214]]]
[[[ 81  79  79]
  [ 73  70  86]
  [ 78  76 108]
  ...
  [ 72  90 120]
  [ 81  95 125]
  [ 78  91 123]]

 [[ 86  81  83]
  [ 79  71  89]
  [ 86  79 111]
  ...
  [ 79  96 128]
  [ 83  96 129]
  [ 86  97 131]]

 [[ 88  83  86]
  [ 87  80  94]
  [ 84  78 104]
  ...
  [ 76  92 125]
  [ 87  97 131]
  [ 87  95 128]]

 ...

 [[ 41  41  28]
  [ 38  39  25]
  [ 38  39  25]
  ..

[[[121 113 101]
  [119 110  99]
  [118 110  99]
  [118 111 100]
  [116 110 100]
  [116 109  98]
  [118 111  97]
  [124 116  99]
  [126 119  98]
  [124 118  93]
  [125 118  96]
  [125 118  98]
  [122 117  97]
  [121 117  97]
  [119 114  95]
  [118 113  95]
  [120 113  99]
  [124 115 104]
  [123 115 102]
  [121 116  99]
  [120 115  97]
  [117 112  94]
  [112 107  88]
  [107 103  83]
  [103 101  81]
  [ 99  99  78]]

 [[120 111 100]
  [120 111 100]
  [118 110  99]
  [116 109  98]
  [114 106  96]
  [115 106  96]
  [115 108  96]
  [119 113 100]
  [122 116 100]
  [124 116  99]
  [125 118  99]
  [122 116  96]
  [120 113  93]
  [122 116  96]
  [123 117 101]
  [123 116 103]
  [123 116 104]
  [127 119 106]
  [126 117 106]
  [126 117 106]
  [121 114 102]
  [111 105  92]
  [108 103  86]
  [106 102  82]
  [103  98  79]
  [103  98  80]]

 [[118 110  99]
  [117 108  97]
  [119 111 100]
  [117 110  99]
  [117 109  98]
  [119 110  99]
  [115 108  97]
  [117 110  99]
  [119 112  98]
  [122 115  97]
  [1

[[[118 111 100]
  [112 105  97]
  [116 108 102]
  [116 112 103]
  [113 114 100]
  [113 115 101]
  [113 114 100]
  [114 111  96]
  [117 112  94]
  [117 112  94]
  [121 116  99]
  [126 120 102]
  [127 120 102]
  [130 121 103]
  [129 117 100]
  [127 115  97]
  [128 116  99]
  [131 122 104]
  [126 118 101]
  [127 119 104]
  [124 115 102]
  [116 108  95]
  [107 101  88]
  [104 100  87]
  [ 96  95  81]
  [ 92  91  78]
  [ 92  90  77]]

 [[118 110 104]
  [117 109 101]
  [121 115 104]
  [120 114 102]
  [120 114 101]
  [119 116 102]
  [114 112  99]
  [116 112  99]
  [119 113 100]
  [121 115 102]
  [120 114 101]
  [126 117 102]
  [121 109  92]
  [120 110  93]
  [118 110  93]
  [120 110  92]
  [122 110  91]
  [124 113  97]
  [124 115 102]
  [122 114 101]
  [121 113 100]
  [117 109  96]
  [109 102  89]
  [104 100  88]
  [ 96  94  83]
  [ 96  92  80]
  [102  96  82]]

 [[121 111 105]
  [121 112 107]
  [120 113 106]
  [117 110 100]
  [123 116 103]
  [124 118 104]
  [120 114 101]
  [117 111  98]
  [1

...

[[[113 114  99]
  [108 108  97]
  [109 108  97]
  ...
  [119 142 171]
  [117 139 171]
  [114 138 170]]

 [[110 112  97]
  [109 110  98]
  [107 107  96]
  ...
  [117 140 170]
  [118 141 171]
  [114 140 169]]

 [[112 112  99]
  [112 112  99]
  [109 110  97]
  ...
  [120 143 168]
  [118 142 167]
  [116 143 170]]

 ...

 [[129 134 167]
  [130 134 168]
  [131 135 169]
  ...
  [126 120 107]
  [125 119 106]
  [123 116 104]]

 [[140 146 175]
  [136 140 172]
  [135 139 171]
  ...
  [129 122 110]
  [125 119 106]
  [124 116 103]]

 [[146 152 178]
  [143 149 173]
  [146 152 178]
  ...
  [130 122 109]
  [125 116 103]
  [126 114 101]]]
[[[114 112  99]
  [114 114  98]
  [112 114  95]
  ...
  [119 142 165]
  [118 142 165]
  [113 141 167]]

 [[116 115 101]
  [116 116 101]
  [115 117  99]
  ...
  [124 147 171]
  [120 144 169]
  [115 140 168]]

 [[118 118 100]
  [117 117  98]
  [117 117  99]
  ...
  [123 149 174]
  [121 149 174]
  [118 146 174]]

 ...

 [[127 133 167]
  [129 135 169]
  [129 135 169]
  ..

[[[168 183 198]
  [171 185 200]
  [175 189 207]
  ...
  [121 145 220]
  [139 160 235]
  [192 209 255]]

 [[166 180 197]
  [168 183 199]
  [171 184 204]
  ...
  [116 135 211]
  [125 141 218]
  [165 177 253]]

 [[163 179 198]
  [165 181 202]
  [164 179 203]
  ...
  [111 130 205]
  [114 132 208]
  [139 156 232]]

 ...

 [[139 148 174]
  [145 151 178]
  [147 151 181]
  ...
  [127 117 111]
  [125 118 112]
  [119 116 110]]

 [[148 156 180]
  [149 157 183]
  [151 159 186]
  ...
  [127 120 113]
  [124 119 112]
  [121 118 111]]

 [[156 164 188]
  [157 165 190]
  [163 171 196]
  ...
  [128 121 115]
  [127 122 116]
  [124 121 115]]]
[[[164 178 201]
  [166 179 203]
  [167 181 206]
  ...
  [111 129 208]
  [119 135 213]
  [134 149 227]]

 [[156 172 191]
  [159 175 196]
  [160 178 201]
  ...
  [110 126 206]
  [114 129 212]
  [122 135 217]]

 [[152 168 187]
  [156 171 193]
  [156 171 194]
  ...
  [109 123 202]
  [120 132 211]
  [123 134 215]]

 ...

 [[145 149 183]
  [149 153 186]
  [150 154 187]
  ..

[[[106 105 109]
  [124 126 135]
  [142 150 165]
  ...
  [ 22  41  35]
  [ 27  44  40]
  [ 33  47  45]]

 [[105 104 106]
  [113 117 121]
  [136 148 162]
  ...
  [ 27  42  36]
  [ 32  47  41]
  [ 35  48  42]]

 [[100 103  98]
  [109 116 115]
  [138 148 160]
  ...
  [ 26  38  33]
  [ 27  41  35]
  [ 33  45  40]]

 ...

 [[118 124 156]
  [124 130 161]
  [126 132 164]
  ...
  [ 55  53  42]
  [ 55  54  42]
  [ 58  56  45]]

 [[121 128 157]
  [125 131 162]
  [125 130 162]
  ...
  [ 55  53  42]
  [ 55  54  42]
  [ 60  58  47]]

 [[120 130 156]
  [136 144 171]
  [135 138 168]
  ...
  [ 60  57  52]
  [ 64  60  55]
  [ 64  61  54]]]
[[[113 116 112]
  [119 126 128]
  [128 138 150]
  ...
  [ 31  43  42]
  [ 37  49  48]
  [ 37  49  48]]

 [[111 114 110]
  [115 121 123]
  [130 138 150]
  ...
  [ 32  39  38]
  [ 40  47  47]
  [ 41  47  47]]

 [[113 109 105]
  [118 117 119]
  [130 134 146]
  ...
  [ 28  36  33]
  [ 38  45  44]
  [ 40  47  47]]

 ...

 [[118 119 161]
  [118 120 160]
  [121 123 163]
  ..

[[[ 81  80  55]
  [ 82  83  58]
  [ 83  82  59]
  ...
  [121 141 174]
  [122 141 174]
  [123 141 174]]

 [[ 85  87  62]
  [ 81  82  57]
  [ 86  84  61]
  ...
  [123 141 174]
  [124 141 174]
  [123 141 174]]

 [[ 90  85  66]
  [ 88  83  63]
  [ 86  84  61]
  ...
  [123 142 175]
  [125 143 176]
  [127 145 177]]

 ...

 [[115 119 142]
  [114 120 144]
  [110 116 141]
  ...
  [ 92  87  70]
  [ 91  85  67]
  [ 92  85  67]]

 [[115 119 142]
  [114 120 144]
  [112 117 142]
  ...
  [ 91  86  68]
  [ 91  85  67]
  [ 90  82  65]]

 [[117 117 141]
  [117 118 143]
  [109 115 141]
  ...
  [ 89  84  66]
  [ 90  84  66]
  [ 89  81  64]]]
[[[ 85  85  64]
  [ 81  81  61]
  [ 81  81  61]
  ...
  [120 140 172]
  [117 138 170]
  [117 139 171]]

 [[ 82  82  63]
  [ 85  84  66]
  [ 88  87  72]
  ...
  [119 141 174]
  [117 139 171]
  [117 139 171]]

 [[ 81  82  70]
  [ 81  82  70]
  [ 81  80  70]
  ...
  [122 144 176]
  [118 140 172]
  [117 139 171]]

 ...

 [[106 114 139]
  [105 113 137]
  [106 114 138]
  ..

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[[[160 180 204]
  [167 186 209]
  [170 186 209]
  ...
  [222 232 255]
  [225 236 255]
  [230 241 255]]

 [[164 181 205]
  [170 185 208]
  [171 185 208]
  ...
  [227 233 255]
  [228 236 255]
  [231 242 255]]

 [[165 180 204]
  [168 183 207]
  [169 183 207]
  ...
  [223 234 255]
  [229 239 255]
  [235 245 255]]

 ...

 [[227 228 252]
  [233 235 255]
  [243 243 255]
  ...
  [ 58  57  62]
  [ 56  55  59]
  [ 55  54  57]]

 [[229 229 252]
  [239 238 255]
  [251 250 255]
  ...
  [ 55  55  55]
  [ 51  51  52]
  [ 54  53  55]]

 [[227 228 251]
  [239 238 255]
  [253 250 255]
  ...
  [ 51  56  51]
  [ 51  54  52]
  [ 56  56  56]]]
[]
[]
[[[147 168 184]
  [150 171 191]
  [150 170 193]
  ...
  [162 179 204]
  [165 179 205]
  [166 181 207]]

 [[146 166 187]
  [148 168 190]
  [151 171 194]
  ...
  [165 180 205]
  [167 182 206]
  [168 184 207]]

 [[148 169 185]
  [149 170 189]
  [151 171